Goals:

- Introduce Polars expressions
- How to write SQL like queries
- Advanced topics - window functions, join, pivot, exploding lists, conditional logic and custom UDFs


In [0]:
!pip install polars

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import polars as pl
import numpy as np
import datetime as dt

In [0]:
# Create a sample dataset to use 

np.random.seed(42)

n = 20
cats = ["A", "B", "C"]

df = pl.DataFrame({
    "id": np.arange(1,n+1),
    "category": np.random.choice(cats, size=n),
    "value": np.random.normal(loc=50, scale=15, size=n).round(2),
    "timestamp": [dt.datetime(2025, 1, 1) + dt.timedelta(days=int(x)) for x in np.random.randint(0, 30, size=n)]
})

# add list column for explode demo

df = df.with_columns([
    pl.int_ranges(1, 1 + (pl.col("id") % 4)).alias("small_list")
])

In [0]:
print(df)

shape: (20, 5)
┌─────┬──────────┬───────┬─────────────────────┬────────────┐
│ id  ┆ category ┆ value ┆ timestamp           ┆ small_list │
│ --- ┆ ---      ┆ ---   ┆ ---                 ┆ ---        │
│ i64 ┆ str      ┆ f64   ┆ datetime[μs]        ┆ list[i64]  │
╞═════╪══════════╪═══════╪═════════════════════╪════════════╡
│ 1   ┆ C        ┆ 43.05 ┆ 2025-01-20 00:00:00 ┆ [1]        │
│ 2   ┆ A        ┆ 43.01 ┆ 2025-01-28 00:00:00 ┆ [1, 2]     │
│ 3   ┆ C        ┆ 53.63 ┆ 2025-01-15 00:00:00 ┆ [1, 2, 3]  │
│ 4   ┆ C        ┆ 21.3  ┆ 2025-01-28 00:00:00 ┆ []         │
│ 5   ┆ A        ┆ 24.13 ┆ 2025-01-07 00:00:00 ┆ [1]        │
│ …   ┆ …        ┆ …     ┆ …                   ┆ …          │
│ 16  ┆ A        ┆ 51.66 ┆ 2025-01-04 00:00:00 ┆ []         │
│ 17  ┆ B        ┆ 32.74 ┆ 2025-01-02 00:00:00 ┆ [1]        │
│ 18  ┆ B        ┆ 55.64 ┆ 2025-01-30 00:00:00 ┆ [1, 2]     │
│ 19  ┆ B        ┆ 40.99 ┆ 2025-01-06 00:00:00 ┆ [1, 2, 3]  │
│ 20  ┆ B        ┆ 45.62 ┆ 2025-01-22 00:00:00 ┆ []    

1. Basic expressions

- pl.col("colname") - references a column
- pl.lit(value) - literal value
- expr.alias("new_name") - rename expression
- expr1 + expr2, expr * 2 etc - arithmetic expressions


In [0]:
(
    df.select([
        pl.col("id"),
        pl.col("value"),
        (pl.col("value") * 1.1).alias("value_plus_10pct"),
        pl.col("category").alias("cat")
    ])
)

id,value,value_plus_10pct,cat
i64,f64,f64,str
1,43.05,47.355,"""C"""
2,43.01,47.311,"""A"""
3,53.63,58.993,"""C"""
4,21.3,23.43,"""C"""
5,24.13,26.543,"""A"""
…,…,…,…
16,51.66,56.826,"""A"""
17,32.74,36.014,"""B"""
18,55.64,61.204,"""B"""


In [0]:
# Using .round() and .floor etc.

(
    df.select([
        pl.col("value").mean().alias("mean_value"),
        pl.col("value").median().alias("median_value"),
        pl.col("value").std().alias("std_value")
    ])
)

mean_value,median_value,std_value
f64,f64,f64
42.406,42.42,12.268675


2. select, with_columns, with_column

- select - creates a new DataFrame with only the provided expressions
- with_columns - adds or replaces multiple columns
- with_column - adds a single column

In [0]:
# Add computed columns using expressions

(
    df.with_columns([
        (pl.col("value") / pl.col("value").sum()).alias("value_share"),
        (pl.col("value").rank("dense")).alias("dense_rank")
    ])
    .select(["id", "category", "value", "value_share", "dense_rank"])
)

id,category,value,value_share,dense_rank
i64,str,f64,f64,u32
1,"""C""",43.05,0.050759,12
2,"""A""",43.01,0.050712,11
3,"""C""",53.63,0.063234,17
4,"""C""",21.3,0.025114,1
5,"""A""",24.13,0.028451,2
…,…,…,…,…
16,"""A""",51.66,0.060911,16
17,"""B""",32.74,0.038603,5
18,"""B""",55.64,0.065604,19


3. Filtering(filter) and boolean logic expressions can be used inside filter

In [0]:
# Filter rows where value > mean

mean_val = df.select(pl.col("value").mean()).item()

high = df.filter(pl.col("value") > mean_val)

print("mean value:", mean_val)
print(high)

mean value: 42.406
shape: (10, 5)
┌─────┬──────────┬───────┬─────────────────────┬────────────┐
│ id  ┆ category ┆ value ┆ timestamp           ┆ small_list │
│ --- ┆ ---      ┆ ---   ┆ ---                 ┆ ---        │
│ i64 ┆ str      ┆ f64   ┆ datetime[μs]        ┆ list[i64]  │
╞═════╪══════════╪═══════╪═════════════════════╪════════════╡
│ 1   ┆ C        ┆ 43.05 ┆ 2025-01-20 00:00:00 ┆ [1]        │
│ 2   ┆ A        ┆ 43.01 ┆ 2025-01-28 00:00:00 ┆ [1, 2]     │
│ 3   ┆ C        ┆ 53.63 ┆ 2025-01-15 00:00:00 ┆ [1, 2, 3]  │
│ 8   ┆ B        ┆ 54.71 ┆ 2025-01-08 00:00:00 ┆ []         │
│ 11  ┆ C        ┆ 71.98 ┆ 2025-01-14 00:00:00 ┆ [1, 2, 3]  │
│ 12  ┆ C        ┆ 46.61 ┆ 2025-01-17 00:00:00 ┆ []         │
│ 13  ┆ A        ┆ 51.01 ┆ 2025-01-04 00:00:00 ┆ [1]        │
│ 16  ┆ A        ┆ 51.66 ┆ 2025-01-04 00:00:00 ┆ []         │
│ 18  ┆ B        ┆ 55.64 ┆ 2025-01-30 00:00:00 ┆ [1, 2]     │
│ 20  ┆ B        ┆ 45.62 ┆ 2025-01-22 00:00:00 ┆ []         │
└─────┴──────────┴───────┴──────────

In [0]:
df.filter((pl.col("category") == "C") & (pl.col("value") > 60))

id,category,value,timestamp,small_list
i64,str,f64,datetime[μs],list[i64]
11,"""C""",71.98,2025-01-14 00:00:00,"[1, 2, 3]"


4. Groupby & Aggregations 

- groupby accepts columns and then use .agg() with expressions


In [0]:
(
    df.group_by("category")
    .agg([
        pl.len().alias("n"),  # calculate number of rows in each category
        pl.col("value").mean().alias("avg_value"), # average value for each category
        pl.col("value").median().alias("median_value"), # median value for each category
        pl.col("value").std().alias("std_value"), # std value for each category
        pl.col("id").max().alias("max_id") # max id for each category
    ])
)

category,n,avg_value,median_value,std_value,max_id
str,u32,f64,f64,f64,i64
"""B""",6,45.255,43.725,8.7627,20
"""C""",9,40.578889,36.38,15.421072,14
"""A""",5,42.276,43.01,11.12044,16


In [0]:
# more than one column in groupby

# avg value and number of rows on each weekday for each category

(
    df.with_columns((pl.col("timestamp").dt.weekday().alias("weekday")))
      .group_by(["category", "weekday"])
      .agg([
          pl.len().alias("n"),
          pl.col("value").mean().alias("avg_value")
      ])
      .sort(["category", "weekday"])

)

category,weekday,n,avg_value
str,i8,u32,f64
"""A""",2,2,33.57
"""A""",6,2,51.335
"""A""",7,1,41.57
"""B""",1,1,40.99
"""B""",3,3,47.386667
…,…,…,…
"""C""",1,1,43.05
"""C""",2,2,46.64
"""C""",3,3,41.606667


For next part:

1. Window functions
2. Joins
3. Pivot
4. List and explode
5. Conditional logic - when/otherwise
6. Lazy dataframe best practices

5. Window functions and rolling aggregrations

- use .over() to apply an expression as a window function
- use groupby_rolling or groupby_dynamic for time based rolling in Polars

In [0]:
# create a simple window: partition by category and compute running mean of 'value'

df_with_rank = (
    df.with_columns([
        pl.col("value").rank("dense").over("category").alias("rank_within_cat"),
        pl.col("value").mean().over("category").alias("mean_within_cat")
    ])
)

df_with_rank

id,category,value,timestamp,small_list,rank_within_cat,mean_within_cat
i64,str,f64,datetime[μs],list[i64],u32,f64
1,"""C""",43.05,2025-01-20 00:00:00,[1],6,40.578889
2,"""A""",43.01,2025-01-28 00:00:00,"[1, 2]",3,42.276
3,"""C""",53.63,2025-01-15 00:00:00,"[1, 2, 3]",8,40.578889
4,"""C""",21.3,2025-01-28 00:00:00,[],1,40.578889
5,"""A""",24.13,2025-01-07 00:00:00,[1],1,42.276
…,…,…,…,…,…,…
16,"""A""",51.66,2025-01-04 00:00:00,[],5,42.276
17,"""B""",32.74,2025-01-02 00:00:00,[1],1,45.255
18,"""B""",55.64,2025-01-30 00:00:00,"[1, 2]",6,45.255


In [0]:
# Rolling / time-based: convert to LazyFrame and use groupby_dynamic for time windows

# create a 7 day window based on timestamp. And then calculate number of rows in each window + avg_value of each window

lf =  (
    df.lazy()
      .with_columns(pl.col("timestamp").cast(pl.Datetime))
      .sort("timestamp")
)

result = (
    lf.group_by_dynamic(
        index_column="timestamp",
        every="7d",
        period="7d",
        offset="0d"
    )
    .agg([
        pl.len().alias("n"),
        pl.col("value").mean().alias("avg_value")
    ])
    .collect()
)

print(result)

shape: (5, 3)
┌─────────────────────┬─────┬───────────┐
│ timestamp           ┆ n   ┆ avg_value │
│ ---                 ┆ --- ┆ ---       │
│ datetime[μs]        ┆ u32 ┆ f64       │
╞═════════════════════╪═════╪═══════════╡
│ 2025-01-02 00:00:00 ┆ 8   ┆ 40.73625  │
│ 2025-01-09 00:00:00 ┆ 4   ┆ 50.89     │
│ 2025-01-16 00:00:00 ┆ 4   ┆ 40.9775   │
│ 2025-01-23 00:00:00 ┆ 3   ┆ 33.04     │
│ 2025-01-30 00:00:00 ┆ 1   ┆ 55.64     │
└─────────────────────┴─────┴───────────┘


6. Joins and concatenations

- Joins in Polars behave SQL like. specify how and on/left_on/right_on
- Use hstack/vstack for concatenation

In [0]:
# create a lookup table for category_metadata

cat_meta = pl.DataFrame({
  "category": ["A","B","C"],
  "description": ["Group A","Group B","Group C"],
  "weight": [1.0, 1.2, 0.9]
})

#join
(df.join(cat_meta, on="category", how="left")).select(["id","category","description","value"])

id,category,description,value
i64,str,str,f64
1,"""C""","""Group C""",43.05
2,"""A""","""Group A""",43.01
3,"""C""","""Group C""",53.63
4,"""C""","""Group C""",21.3
5,"""A""","""Group A""",24.13
…,…,…,…
16,"""A""","""Group A""",51.66
17,"""B""","""Group B""",32.74
18,"""B""","""Group B""",55.64


In [0]:
#hstack and vstack for concatenation

#hstack

df1 = pl.DataFrame({
    "id" : [1, 2, 3],
    "category": ["A", "B", "C"]
})

df2 = pl.DataFrame({
    "value":[10, 20, 30]
})

df_hstack = df1.hstack(df2)

print(df_hstack)

shape: (3, 3)
┌─────┬──────────┬───────┐
│ id  ┆ category ┆ value │
│ --- ┆ ---      ┆ ---   │
│ i64 ┆ str      ┆ i64   │
╞═════╪══════════╪═══════╡
│ 1   ┆ A        ┆ 10    │
│ 2   ┆ B        ┆ 20    │
│ 3   ┆ C        ┆ 30    │
└─────┴──────────┴───────┘


In [0]:
#vstack - vertical concatenation

df2 = pl.DataFrame({
    "id" : [4, 5],
    "category": ["A", "B"]
})

df_vstack = df1.vstack(df2)

print(df_vstack)

shape: (5, 2)
┌─────┬──────────┐
│ id  ┆ category │
│ --- ┆ ---      │
│ i64 ┆ str      │
╞═════╪══════════╡
│ 1   ┆ A        │
│ 2   ┆ B        │
│ 3   ┆ C        │
│ 4   ┆ A        │
│ 5   ┆ B        │
└─────┴──────────┘


7. Pivot and Unpivot(melt)

- pivot - useful for turning groupby aggregations to wide form
- melt - (unpivot) from wide to long

In [0]:
# pivot: average value per category for each weekday

(
    df.with_columns(pl.col("timestamp").dt.weekday().alias("weekday"))
      .pivot(
          values= "value",
          index= "weekday",
          on="category",
          aggregate_function="mean"
      )
)

weekday,C,A,B
i8,f64,f64,f64
1,43.05,null,40.99
2,46.64,33.57,null
3,41.606667,null,47.386667
7,null,41.57,null
5,37.715,null,null
6,28.63,51.335,null
4,null,null,44.19


In [0]:
# melt (unpivot) 

wide = pl.DataFrame({
    "id": [1, 2, 3],
    "A": [10, 20, 30],
    "B": [5, 6, 7]
})

wide.unpivot(index=["id"], on=["A","B"], variable_name="category", value_name="value")

id,category,value
i64,str,i64
1,"""A""",10
2,"""A""",20
3,"""A""",30
1,"""B""",5
2,"""B""",6
3,"""B""",7


8. Working with list and explode

- Polars has native list-columns and many list functions.

In [0]:
# explode the list column values into separate rows

exploded = df.explode("small_list")
print(exploded)

shape: (35, 5)
┌─────┬──────────┬───────┬─────────────────────┬────────────┐
│ id  ┆ category ┆ value ┆ timestamp           ┆ small_list │
│ --- ┆ ---      ┆ ---   ┆ ---                 ┆ ---        │
│ i64 ┆ str      ┆ f64   ┆ datetime[μs]        ┆ i64        │
╞═════╪══════════╪═══════╪═════════════════════╪════════════╡
│ 1   ┆ C        ┆ 43.05 ┆ 2025-01-20 00:00:00 ┆ 1          │
│ 2   ┆ A        ┆ 43.01 ┆ 2025-01-28 00:00:00 ┆ 1          │
│ 2   ┆ A        ┆ 43.01 ┆ 2025-01-28 00:00:00 ┆ 2          │
│ 3   ┆ C        ┆ 53.63 ┆ 2025-01-15 00:00:00 ┆ 1          │
│ 3   ┆ C        ┆ 53.63 ┆ 2025-01-15 00:00:00 ┆ 2          │
│ …   ┆ …        ┆ …     ┆ …                   ┆ …          │
│ 18  ┆ B        ┆ 55.64 ┆ 2025-01-30 00:00:00 ┆ 2          │
│ 19  ┆ B        ┆ 40.99 ┆ 2025-01-06 00:00:00 ┆ 1          │
│ 19  ┆ B        ┆ 40.99 ┆ 2025-01-06 00:00:00 ┆ 2          │
│ 19  ┆ B        ┆ 40.99 ┆ 2025-01-06 00:00:00 ┆ 3          │
│ 20  ┆ B        ┆ 45.62 ┆ 2025-01-22 00:00:00 ┆ null  

9. Conditional logic : when/otherwise

- similar to case when logic in SQL
- pl.when(condition).then(expr).otherwise(expr)

In [0]:
# CASES : when value>60 then high, >40 ->medium, otherwise low

(
  df.with_columns([
    pl.when(pl.col("value") > 60).then(pl.lit("high"))
      .when(pl.col("value") > 40).then(pl.lit("medium"))
      .otherwise(pl.lit("low")).alias("value_bucket")
  ])
)

id,category,value,timestamp,small_list,value_bucket
i64,str,f64,datetime[μs],list[i64],str
1,"""C""",43.05,2025-01-20 00:00:00,[1],"""medium"""
2,"""A""",43.01,2025-01-28 00:00:00,"[1, 2]","""medium"""
3,"""C""",53.63,2025-01-15 00:00:00,"[1, 2, 3]","""medium"""
4,"""C""",21.3,2025-01-28 00:00:00,[],"""low"""
5,"""A""",24.13,2025-01-07 00:00:00,[1],"""low"""
…,…,…,…,…,…
16,"""A""",51.66,2025-01-04 00:00:00,[],"""medium"""
17,"""B""",32.74,2025-01-02 00:00:00,[1],"""low"""
18,"""B""",55.64,2025-01-30 00:00:00,"[1, 2]","""medium"""


Advanced example

Q. list top-2 values per category

A. approach - rank + window function using over partition. Filter only top 2 in each category

In [0]:
ranked = (
    df.with_columns(
        pl.col("value").rank("dense").over("category").alias("rank_within_cat")
    )
    .filter(pl.col("rank_within_cat") <= 2)
    .sort(["category", "rank_within_cat"], descending=[False,False])
)

print(ranked)

shape: (6, 6)
┌─────┬──────────┬───────┬─────────────────────┬────────────┬─────────────────┐
│ id  ┆ category ┆ value ┆ timestamp           ┆ small_list ┆ rank_within_cat │
│ --- ┆ ---      ┆ ---   ┆ ---                 ┆ ---        ┆ ---             │
│ i64 ┆ str      ┆ f64   ┆ datetime[μs]        ┆ list[i64]  ┆ u32             │
╞═════╪══════════╪═══════╪═════════════════════╪════════════╪═════════════════╡
│ 5   ┆ A        ┆ 24.13 ┆ 2025-01-07 00:00:00 ┆ [1]        ┆ 1               │
│ 6   ┆ A        ┆ 41.57 ┆ 2025-01-12 00:00:00 ┆ [1, 2]     ┆ 2               │
│ 17  ┆ B        ┆ 32.74 ┆ 2025-01-02 00:00:00 ┆ [1]        ┆ 1               │
│ 19  ┆ B        ┆ 40.99 ┆ 2025-01-06 00:00:00 ┆ [1, 2, 3]  ┆ 2               │
│ 4   ┆ C        ┆ 21.3  ┆ 2025-01-28 00:00:00 ┆ []         ┆ 1               │
│ 14  ┆ C        ┆ 28.63 ┆ 2025-01-18 00:00:00 ┆ [1, 2]     ┆ 2               │
└─────┴──────────┴───────┴─────────────────────┴────────────┴─────────────────┘


In [0]:
# compute percentiles per group (75th percentile). Use quantile

(
    df.group_by("category")
      .agg([
          pl.col("value").quantile(0.75).alias("p75"),
          pl.col("value").quantile(0.5).alias("median")
      ])
)

category,p75,median
str,f64,f64
"""C""",46.61,36.38
"""B""",54.71,45.62
"""A""",51.01,43.01


10. Write SQL like expressions



example: 
SQL:

# when value is >20, calculate avg value per category. Then show only those entries where  avg_value > 45 in the order of avg_value descending and limit to 10 entries

SELECT 
  category, 
  avg(value) as avg_value
FROM df
  WHERE value>20
GROUP BY category
HAVING AVG(value)>45
ORDER BY avg_value DESC
LIMIT 10

In [0]:
(
    df.lazy()
      .filter(pl.col("value") > 20)
      .group_by("category")
      .agg(pl.col("value").mean().alias("avg_value"))
      .filter(pl.col("avg_value") > 45)
      .sort("avg_value", descending = True)
      .limit(10)
      .collect()
)

category,avg_value
str,f64
"""B""",45.255


Best practices in Polars

1. Prefer lazyframe for large pipelines. df.lazy() - > expressions - >collect()
2. use with_columns instead of repeated .with_column (fewer passes)
3. Avoid .apply() and Python level UDFs. Use verctorized expressions more
4. Use select to prune unwanted columns
